In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import openai
import google.genai as genai
import os

In [ ]:
# Load patient education data
df = pd.read_csv('/content/medquad.csv')
df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [ ]:

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed all content once
corpus_embeddings = model.encode(df['answer'].head(100).tolist(), convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:


GEMINI_AI_API_KEY = os.getenv('GEMINI_AI_API_KEY')
gemmini_client = genai.Client(api_key=GEMINI_AI_API_KEY)
gemmini_client = genai.Client(api_key=GEMINI_AI_API_KEY)

In [ ]:
df = df.head(100)
def find_best_resources(query, top_k=3):
    query_embedding = model.encode(query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = cosine_scores.topk(k=top_k)

    results = []
    for score, idx in zip(top_results.values, top_results.indices):
        idx=idx.item()
        title = df.iloc[idx]['question']
        content = df.iloc[idx]['answer']

        results.append((title, content, score.item()))
    return results

from openai import OpenAI

import os
from openai import OpenAI



def generate_friendly_response(resources):
    prompt = "You are a friendly healthcare assistant. Please explain these topics as brief bullet points in simple language with emojis where appropriate:\n\n"
    for i, (title, content, _) in enumerate(resources, 1):
        prompt += f"{i}. {title}: {content}\n"
    prompt += "\nPlease provide a concise, easy-to-understand summary with bullet points and emojis."

    response = gemmini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    info = response.text.strip()
    return info


def chat():
    print("Welcome to the Patient Education Chatbot! Type your question or 'exit' to quit.")
    while True:
        user_input = input("\nYour question: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        best_resources = find_best_resources(user_input)
        reply = generate_friendly_response(best_resources)
        print("\nBot:", reply)

chat()

Welcome to the Patient Education Chatbot! Type your question or 'exit' to quit.

Your question: How can I prevent heart disease?

Bot: Okay, here's a breakdown of those topics in simple bullet points with emojis:

**1. Preventing High Blood Pressure: Healthy Habits**

*   **Eat Healthy:** 🍎 Lots of fruits, veggies, low-fat dairy, whole grains. Less salt, saturated fat, and cholesterol. Try the DASH diet!
*   **Lower Salt Intake:** 🧂 Read labels, avoid processed foods, fast food.
*   **Get Active:** 🏃‍♀️ Aim for at least 2.5 hours of moderate exercise per week.
*   **Healthy Weight:** ⚖️ Maintain a healthy weight, even losing a little can help!
*   **Limit Alcohol:** 🍷 No more than 1 drink/day for women, 2 for men.
*   **Don't Smoke:** 🚭 Quit smoking! It damages your blood vessels.
*   **Manage Stress:** 🧘‍♀️ Find healthy ways to cope with stress (yoga, meditation).

**2. Preventing High Blood Pressure: More Tips**

*   **DASH Diet:** 🥗 Eat lots of fruits, vegetables, whole grains, and 